In [1]:
!pip install -U yt-dlp -q
!pip install transformers datasets evaluate jiwer -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.0 MB/s eta 0:00:00


## Скачивание данных

In [2]:
import torch
import torchaudio

In [3]:
link_whales = 'https://www.youtube.com/watch?v=bgiPTUy2RqI'
link_penguins = 'https://www.youtube.com/watch?v=q3uXXh1sHcI'

In [ ]:
!yt-dlp -x --verbose --write-auto-subs --audio-format wav {link_whales}

In [5]:
audio_path = '/content/Blue Whales 101 ｜ Nat Geo Wild [bgiPTUy2RqI].wav'
subtitle_path = '/content/Blue Whales 101 ｜ Nat Geo Wild [bgiPTUy2RqI].en.vtt'

### Подготовка субтитров с YouTube

Так как субтитры разделены по по временным отрезкам, имеет смысл их с этими отрезками сохранить. Таким образом мы получаем временные ограничения для фраз, то есть мы с большей уверенностью при делении аудио на куски не разделим слова на две части. Делить аудио на отрезки в дальнейшем будет удобно при подаче аудио на вход модели, так как при обработке большого аудио требуется много оперативной памяти, а таким богаством мы пока не владеем :(

In [6]:
import re

In [7]:
with open(subtitle_path, 'r') as f:
    subtitles = f.read()

In [8]:
# регулярка, чтобы достать слова из строки
phrase_finder = re.compile(r'<c> (\w+?)<\/c>')

In [9]:
# функция перевода времени в секунды
def turn_to_seconds(timestamp):
    hours, minutes, seconds = timestamp.split(':')
    return int(hours) * 3600 + int(minutes) * 60 + float(seconds)

timestamps = {}
for part in subtitles.split('\n\n')[1:]:
    try:
        timestamp, _, phrase = part.split('\n')
        start, end = timestamp.split('-->')
        start = turn_to_seconds(start)
        end = turn_to_seconds(end.split(' ')[1])

        phrase = [phrase.split('<')[0]] + phrase_finder.findall(phrase)
        if (phrase != [' ']) and (phrase[0][0] != '['):
            timestamps[(start, end)] = ' '.join(phrase)
    except:
        # пустая строка
        print('error', part)

error 


In [10]:
for key, value in list(timestamps.items())[:5]:
    print(key)
    print(value)
    print()

(5.27, 8.6)
blue whales are the largest animals to

(8.61, 12.499)
have ever existed as long as about three

(12.509, 15.77)
school buses and as heavy as 15 blue

(15.78, 19.64)
whales even outsized the dinosaurs the

(19.65, 22.07)
whales marine habitat contributes to



In [20]:
youtube_text = ' '.join(list(timestamps.values())).lower()

### Эталон

In [11]:
import json

In [13]:
etalon_path = '/content/drive/MyDrive/NLP Course/etalon.json'
with open(etalon_path, 'r') as f:
    etalon = f.read()
etalon = json.loads(etalon)

In [14]:
etalon = {(float(time.split('-')[0]), float(time.split('-')[1])): phrase for time, phrase in etalon.items()}

In [15]:
for key, value in list(etalon.items())[:5]:
    print(key)
    print(value)
    print()

(5.27, 8.6)
blue whales are the largest animals to

(8.61, 12.499)
have ever existed as long as about three

(12.509, 15.77)
school buses and as heavy as 15 blue

(15.78, 19.64)
whales even outsized the dinosaurs the

(19.65, 22.07)
whales marine habitat contributes to



In [21]:
etalon_text = ' '.join(list(etalon.values())).lower()

### Подготовка аудио

Изначально аудио имеет частоту дискретизации 48000. Для моделей подходит частота 16000, поэтому необходимо ресэмплировать. Также аудио получилось в стереоформате - беру среднее, чтобы получить моно.

In [16]:
waveform, sample_rate = torchaudio.load(audio_path)
waveform_sr = torchaudio.functional.resample(waveform, orig_freq=sample_rate, new_freq=16_000)
waveform_mono = waveform_sr.mean(dim=0)

## 1. Transformers from Hugging Face

### Wav2Vec

In [17]:
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [26]:
from tqdm import tqdm

# функция проходится по каждому отрезку времени
# add_time работает как небольшая погрешность,
# чтобы слова были услышаны полностью
def get_asr_subtitles(timestamps, add_time=800, sr=16_000):
    asr_phrases = {}
    for key in tqdm(timestamps.keys()):
        start_index = int(key[0] * sr) - add_time
        end_index = int(key[1] * sr) + add_time
        text = transcriber(waveform_mono[start_index:end_index].numpy())['text']
        asr_phrases[key] = text
    return asr_phrases

asr_phrases = get_asr_subtitles(timestamps)

Так как используется параметр add_time, при расшифровке аудио отрезки наклыдваются, поэтому какие-то расшифровки могут начинаться на символы/слова, на которые заканчиваются предыдущие. Это нужно учитывать, когда отрезки склеиваются в один текст. Большинство таких перекрытий покрывает функция text_from_phrases

In [35]:
def text_from_phrases(phrases):
    text = list(phrases.values())[0].lower()
    for phrase in list(phrases.values())[1:]:
        phrase = phrase.lower()
        if (text[-1] == phrase[0]):
            text += phrase[1:]
        elif text[-2:] == phrase[:2]:
            text += phrase[2:]
        elif text[-3:] == phrase[:3]:
            text += phrase[3:]
        elif text.split()[-1] == phrase.split()[0]:
            text += ' ' + ' '.join(phrase.split()[1:])
        else:
            text += f' {phrase}'
    return text

wav2vec_text = text_from_phrases(asr_phrases)

In [ ]:
wav2vec_text

### Whisper

In [29]:
transcriber = pipeline("automatic-speech-recognition", "facebook/hubert-xlarge-ls960-ft")
hubert_phrases_1500 = get_asr_subtitles(timestamps, add_time=1500)

Some weights of the model checkpoint at facebook/hubert-xlarge-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-xlarge-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a dow

In [37]:
hubert_text = text_from_phrases(hubert_phrases_1500)

## Speech Recognition

In [40]:
!pip install SpeechRecognition -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 15.9 MB/s eta 0:00:00


### Google API

In [41]:
import speech_recognition as spr

recognizer = spr.Recognizer()

whales = spr.AudioFile(audio_path)
with whales as source:
    audio = recognizer.record(source)

In [42]:
google_text = recognizer.recognize_google(audio)

## Оценивание

In [44]:
from jiwer import wer

wav2vec_wer = wer(etalon_text, wav2vec_text)
youtube_wer = wer(etalon_text, youtube_text)
google_wer = wer(etalon_text, google_text)
hubert_wer = wer(etalon_text, hubert_text)

In [46]:
print('youtube_wer:', youtube_wer)
print('wav2vec_wer:', wav2vec_wer)
print('google_wer:', google_wer)
print('hubert_wer:', hubert_wer)

youtube_wer: 0.009302325581395349
wav2vec_wer: 0.23255813953488372
google_wer: 0.12790697674418605
hubert_wer: 0.1744186046511628


Таким образом, я сравнила автоматические субтитры с YouTube с работой трёх моделей распознавания речи: Wav2Vec и Hubert из библиотеки transformers и GoogleAPI из библиотеки speech_recognition. Лучше всего себя показали автоматические субтитры YouTube, на втором месте результаты работы API от Google, после - Hubert. Хуже всего себя показала модель Wav2Vec. Вероятнее всего, это произошло из-за того, что я использовала базовую модель Wav2Vec, а Hubert - большую, поэтому Hubert сумел выучить чуть больше полезной информации.